# <div align="center">Detrital Common Pb Corrections</div>
## <div align="center">Python code for common Pb corrections of detrital U-Pb data</div>
#### <div align="center">Megan Mueller</div>
##### <div align="center">Please cite the accompanying article under consideration in Geochronology:</div>
<div align="center">Mueller et al., https://doi.org/10.5194/egusphere-2023-1293 </div>



<br><div align="center">Version 1.1.3</div>

# Overview 
This notebook aims to perform 207Pb and 208Pb corrections for detrital U-Pb data. 

### Initial Conditions
The code was written using file input as .xlsx file from iolite 4 output, described below. The code can be modified for other file input structures. 

### Organization
The notebook is divided into the following sections: 
1. [Data import and initial code](#Section1)
2. [208Pb correction](#Section2)
3. [207Pb correction](#Section3)
4. [Export results](#Section4)


<a id='Section1'></a>
# 1. Import Data and Set-Up


## 1.1 Install Python Packages and Import Libraries

In [ ]:
### Install Python packages
# Run to see if installed, if not, will install
# Only need to install once

import importlib
from subprocess import run

def install_if_not_installed(package_name):
    try:
        importlib.import_module(package_name)
        print(f"Package {package_name} is already installed.")
    except ImportError:
        print(f"Installing package: {package_name}")
        run(['pip', 'install', package_name], check=True)
        print(f"Installed package: {package_name}")

install_if_not_installed("pandas") # Install pandas
install_if_not_installed("numpy") # Install numpy
install_if_not_installed("matplotlib") # Install matplotlib
install_if_not_installed("tabulate") # Install tabulate
install_if_not_installed("sympy") # Install sympy
install_if_not_installed("easygui") # Install easygui
install_if_not_installed("cmcrameri") # Install cmcrameri


In [ ]:
### Import necessary libraries

import os
import numpy as np
from matplotlib import cm
from sympy import symbols, Eq, solve, exp
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
from scipy.optimize import root
import pandas as pd
import matplotlib.pyplot as plt
#import matplotlib.ticker as ticker
from tabulate import tabulate
import tkinter as tk
from tkinter import filedialog
import easygui

from cmcrameri import cm


%matplotlib inline
%matplotlib notebook

## 1.2 Input File Organization (Data Needed to Run)
Run this section of code to load Excel spreadsheet with U-Pb data. Edit the code to specify the orrect file path, file name, and Excel sheet tab.
The following sections of code use the following data which must be present in the input file: 
- Grain ID name
- Counts per second: 206, 207, 208
- U-Pb ratios and uncertainties (1s or 2s absolute): 206/238, 238/206 and 207/206
- Error correlation (rho 207Pb/206Pb v 238U/206Pb)
- U/Th ratio
- U concentration (ppm)


The example dataset is a table of U-Pb and trace element data for all detrital rutile unknowns from Mueller et al.:

ExampleDataset-DetritalCommonPbCorrections-Muelleretal_Unknowns


In [ ]:
### Load Excel file with data into DataFrame

# Specify the folder and file name
folder_path = r'C:\Users\megan\Dropbox\CommonPbPython\Example-data'  # Update this with the actual path
file_name = 'ExampleDataset-DetritalCommonPbCorrections-Muelleretal_Unknowns.xlsx'  # Update this with the actual file name
sheet_name = 'AllUnknowns'  # Update this with the actual sheet name

# Combine folder path and file name using os.path.join
excel_file_path = os.path.join(folder_path, file_name)

# Load a specific sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Display the DataFrame
pd.set_option('display.max_columns', None)
# df
df.head(10) # display first 10 rows of DataFrame


<a id='DefineVariables'></a>
## 1.3 Define Variables
The following cell defines the input variables used in all subsequent calculations. Note that the column headings are displayed in the output of the above cell. Change the below code if the columns are named differently. Code here uses default column headings from iolite 4 export.

Note that all calculations are assuming the input uncertainties are 2 sigma absolute. The cell below prompts the user to specify whether inputs are 1s absolute or 2s absolute; if 1s absolute then the inputs are converted to 2s absolute. The below code expects uncertainty values as absolute not percent.

*Warning: Code prompts user (bottom), will not continue without an input*

In [ ]:
### Edit this section to add/remove columns to include as variables as needed
# Everything here will be used in subsequent code
# Edit if column names are different than below. See column names in above cell output (or, run df.head(15) )

# Define variables by column headers
data_dict = df.to_dict(orient='series')

# Initial measured and calculated values (required)
GrainID = df['Grain_ID']
i206CPS = df['Pb206_CPS_mean']
i207CPS = df['Pb207_CPS_mean']
i208CPS = df['Pb208_CPS_mean']
iUTh = df['Raw U/Th_mean']
i386 = df['Final U238/Pb206_mean']
i386err = df['Final U238/Pb206_2SE(prop)'] # 2s absolute
i638 = df['Final Pb206/U238_mean']
i638err = df['Final Pb206/U238_2SE(prop)'] # 2s absolute
t638 = df['Final Pb206/U238 age_mean']
t638err = df['Final Pb206/U238 age_2SE(prop)'] # 2s absolute
i76 = df['Final Pb207/Pb206_mean']
i76err = df['Final Pb207/Pb206_2SE(prop)'] # 2s absolute
t76 = df['Final Pb207/Pb206 age_mean']
t76err = df['Final Pb207/Pb206 age_2SE(prop)'] # 2s absolute
rho = df['rho 207Pb/206Pb v 238U/206Pb']
U = df['Approx_U_PPM_mean']

user_input = input("\033[1mAre input file uncertainties 1-sigma absolute or 2-sigma absolute? \033[0m\n Type '1' or '2' (not 'one' or 'two') \n If 2, no calculations will be performed \n If 1, input uncertainties will be multiplied by 2 \n")
if user_input.lower() == '2':
    print("\033[1mInput uncertainties are 2s\033[0m")
elif user_input == '1':
    # Will run the following code if input uncertainties are 1s (abs) and not 2s (abs)
    i386err = df['Final U238/Pb206_2SE(prop)'] * 2 # 2s absolute
    i638err = df['Final Pb206/U238_2SE(prop)'] * 2 # 2s absolute
    t638err = df['Final Pb206/U238 age_2SE(prop)'] * 2 # 2s absolute
    i76err = df['Final Pb207/Pb206_2SE(prop)'] * 2 # 2s absolute
    t76err = df['Final Pb207/Pb206 age_2SE(prop)'] * 2 # 2s absolute
    print('\033[1mReminder:\033[0m Input uncertainties are 1s and were multiplied by 2. Subsequent calculations and outputs will be at 2s level')
else:
    print("\033[1mOops. Invalid input. Please enter '1' or '2'.\033[0m")

In [ ]:
# Initial calculations for Pb corrections

# Define constants
instUTh = 1; # instrument-specific U/Th, should be =1 and already corrected during data reduction
rat85 = 137.88; # 238U/235U ratio. 137.88 from Steiger and Jäger (1977). 137.818 from Hiess et al. (2012). 
rat58 = 0.007252683; # 1/137.88 constant 235/238 U ratio
lambda238 = 1.55125E-10; # U238 decay constant from Faure (1986)
lambda235 = 9.8485E-10; # U235 decay constant from Faure (1986)
lambda232 = 4.9475E-11; # 232Th decay constant

# Initialize an array for ratios
R = np.zeros((len(GrainID), 5))

# Calculate ratios
# Avoid division by zero and handle invalid values
meas68 = np.divide(i206CPS, i208CPS, out=np.zeros_like(i206CPS), where=i208CPS != 0)
instThUc = np.divide(1, iUTh, out=np.zeros_like(iUTh), where=iUTh != 0) * instUTh
meas67 = np.divide(i206CPS, i207CPS, out=np.zeros_like(i206CPS), where=i207CPS != 0)
meas76 = np.divide(i207CPS, i206CPS, out=np.zeros_like(i207CPS), where=i206CPS != 0)

# Create the R array with all ratios for each grain
R = np.column_stack((meas68, 1/meas68, instThUc, meas67, meas76))

# Display the calculated ratios
#print("\033[1mCalculated Ratios (Stored in R):\033[0m")
#print(R)

<a id='Section2'></a>
# 2. 208Pb Correction
Iteratively calculate 208Pb corrected age

The 208Pb correction method determines the common Pb component using the 232Th-208Pb decay scheme and assumes U-Th-Pb concordance, Th/U remained undistrubed, and no Pb loss. Because Pb loss is not considered, all corrected dates are (possibly) minimum ages.

The 208Pb correction iterates on initial age and Stacey and Kramers (1975) terrestrial Pb evolution model. The measured isotopes are the iolite baseline-corrected CPS values (defined in [Section 1](#DefineVariables)). The initial age is the uncorrected date, which is then replaced by the result of subsequent iterations. For the 2 stage terrestrial Pb evolution model of Stacey and Kramers (1975, EPSL), 3700 Ma is the boundary between the first and second stages. If the initial date is > 4570 Ma, it is set to 4750 Ma.

The method here directly follows what is outlined in published papers: Williams (1997), Chew et al. (2011), McLean et al. (2011), Odlum et al. (2019), and Vermeesch (2020). The equations below benefitted from inital discussion with Margo Odlum and the UTChron group.

## 2.1 Define 208Pb Correction Function

In [ ]:
# Define function for 208 Pb corrections
# Only need to run this once per session

# 208 Pb Correction
def iterative_208Pb(initial_age, i638_values, i638err_values):
    result_array = np.zeros((len(initial_age), 8))

    # Iterative section for 208Pb Correction
    for i in range(len(initial_age)):
        # Calculate initial common Pb section
        # 1st stage Pb evolution from Stacey & Kramers 1975 EPSL (Table 8)
        if (initial_age[i] != 0.0) & (~np.isnan(initial_age[i])): # Check for NaN or zero values in initial_age
            if initial_age[i] > 3700:
                a64 = 7.19 * (np.exp(lambda238 * 4.570E9) - np.exp(lambda238 * initial_age[i] * 1000000)) + 9.307
                a84 = 33.21 * (np.exp(lambda232 * 4.570E9) - np.exp(lambda232 * initial_age[i] * 1000000)) + 29.487
                a86 = a84 / a64 # 208/206 common
            else:
                # 2nd stage Pb evolution from Stacey & Kramers 1975 EPSL (Eqns 5-6; Table 8); see also McLean et al 2011 G3 (Eqns A28-A30); and Vermeesch 2020 Geochron (Eqns 25-26)
                a64 = 9.74 * (np.exp((lambda238 * 3.7E9)) - np.exp((lambda238 * initial_age[i] * 1000000))) + 11.152 # 206/204 common ratio from estimated dates younger than 3.7 Ga using Stacey-Kramers eqn. and initial 206/238 Age estimate
                a84 = 36.84 * (np.exp((lambda232 * 3.7E9)) - np.exp((lambda232 * initial_age[i] * 1000000))) + 31.23 # 208/204 common ratio from estimated dates younger than 3.7 Ga using Stacey-Kramers eqn. and initial 206/238 Age estimate
                a86 = a84 / a64 # 208/206 common
        else:
            pass

        # 208 Pb correction
        if i638[i] != 0.0: # Check for zero values in i638[i]
            # 208 Pb correction
            # calculate expected radiogenic Pb ratios: 208Pb* / 206Pb* = (Th/U) [(eλ232 t* -1) / (eλ238 t* -1)]
            c = (np.exp(lambda232 * (initial_age[i] * 1000000))) - 1 # 208Pb produced from initial age estimate
            d = (np.exp(lambda238 * (initial_age[i] * 1000000))) - 1 # 206Pb produced from initial age estimate
            e = R[i, 2] # recall corrected ThU value
            c86Pb = e * (c / d) # expected radiogenic Pb 208Pb* / 206Pb*

            # calculate f206, fraction of common Pb
            g = R[i, 1] # recall measured 208/206 Pb ratio calculated from CPS
            f206 = (g - c86Pb) / (a86 - c86Pb) # f206 = (meas. - expected)/(common - expected)
            Rad638 = (1 - f206) * i638[i] # Radiogenic component, 206*/238 ratio 

            # solve age equation with corrected radiogenic 206*/238 ratio to get the corrected age: t = ln ( 206/238 + 1 ) / λ238
            AgeTot206 = ((np.log(i638[i] + 1)) / lambda238) / 1000000 # age with no correction, total 206
            Age208c = ((np.log(np.abs(Rad638) + 1)) / lambda238) / 1000000 ## *** Corrected 208 age *** 
            AgeDiffCor = Age208c - AgeTot206  # difference in ages

            # calculate 2SE abs error of 208corrected age using initial % error
            p = i638[i] # recall initial Final 206/238 ratio
            q = i638err[i] #recall initial Final 206/238 uncertainty 2SE abs
            cAge208cErr = Age208c / 100 * (q / p * 100) # calculate 2SE abs error of corrected age using initial % uncertainty

           # Store results in the array
            result_array[i, :] = [a86, c86Pb, f206, Rad638, AgeTot206, Age208c, cAge208cErr, AgeDiffCor]

        else:
            pass # Handle the case where i638[i] is zero

    return result_array

#print('Complete')

## 2.2 Perform Iterative 208Pb Correction
<a id='208Manual'></a>
### 2.2.1 Iterative 208Pb Correction - the number of iterations set manually
The cell below runs the 208Pb correction. Define the desired number of iterations below. At least 5 is recommended.

In [ ]:
# 208Pb Correction 

# Define number of iterations
num_iterations = 5

# Initialize a list to store DataFrames
dfs208_list = []

# Initial date estimate
date_estimate = t638 # iteration 1 uses uncorrected date, subsequent iterations use date calculated in previous iteration
date_estimate = np.where(date_estimate > 4570, 4570, date_estimate)

# Perform iterations
for iteration in range(1, num_iterations + 1):
    # Perform 208Pb correction for the current iteration
    current_iteration_results = iterative_208Pb(date_estimate, i638, i638err)
    
    # Create a DataFrame for the current iteration results
    df_iteration = pd.DataFrame(
        current_iteration_results,
        columns=[f"208It{iteration}_208/206c", f"208It{iteration}_208/206*",
                 f"208It{iteration}_f206", f"208It{iteration}_206*/238",
                 f"208It{iteration}_DateTot206", f"208It{iteration}_Date208c",
                 f"208It{iteration}_Date208cErr", f"208It{iteration}_AgeDiffCorr"]
    )
    
    # Print information about the current iteration
    print(f'\n\033[1m207Pb Correction - Iteration {iteration} Complete\033[0m')
    print('Date of grain in last row (Ma):', current_iteration_results[-1, 5])
    
    # Append the DataFrame to the list
    dfs208_list.append(df_iteration)
    
    # Update the date_estimate for the next iteration
    date_estimate = current_iteration_results[:, 5]

# Display the DataFrames
print('\n\n *Preview of results from every iteration:* \n')
for i, result208_df in enumerate(dfs208_list, start=1):
    print(f'\n\n\033[1m207Pb Correction - Iteration {i} Results DataFrame:\033[0m')
    print('\ncolumns: 1- 208/206 common, 2- 208/206* radiogenic, 3- Fraction common Pb (f206), 4- 206*/238 ratio, 5- Total 206 Date (Ma), 6- 208Pb-corrected date (Ma), 7- 208Pb-corrected date uncertainty (2s abs), 8- Difference in 208Pb corr from total206 dates (Ma) \n\n')
    print(result208_df.to_string(max_rows=15))
    

# Combine DataFrames along the second axis (columns)
#dfs_list.append(df) # append input dataframe to iteration output dataframes
#result_df208 = pd.concat([df, df_It208_1, df_It208_2, df_It208_3, df_It208_4, df_It208_5], axis=1)
result208_df = pd.concat([df] + dfs208_list, axis=1)    

<a id='208Threshold'></a>
### 2.2.2 Iterative 208Pb correction - number of iterations set by threshold

Loops iterations until either the threshold or num_iterations is reached. The threshold is defined as the percent difference in the corrected date between the current and previous iteration (i.e., 1%). If the date difference is above the threshold for any grains, another iteration will run for all grains.

Note: All iteration results are displayed but only the first and last are included in the export table (Sections 2.3 and 2.4)

In [ ]:
# Define the number of iterations
num_iterations = 200  # Set a large number to ensure it runs until the threshold is met, may need to increase

# Specify the threshold
threshold = 1 / 100  # ex: 1% expressed as a decimal

# Initialize a list to store DataFrames
dfs208_list = []

# Initial date estimate
date_estimate = t638  # iteration 1 uses uncorrected date, subsequent iterations use date calculated in the previous iteration
date_estimate = np.where(date_estimate > 4570, 4570, date_estimate)

# Initialize iteration counter
iteration = 1

# Perform iterations
while iteration <= num_iterations:
    # Perform 208Pb correction for the current iteration
    current_iteration_results = iterative_208Pb(date_estimate, i638, i638err)
    
    # Create a DataFrame for the current iteration results
    df_iteration = pd.DataFrame(
        current_iteration_results,
        columns=[f"208It{iteration}_208/206c", f"208It{iteration}_208/206*",
                 f"208It{iteration}_f206", f"208It{iteration}_206*/238",
                 f"208It{iteration}_DateTot206", f"208It{iteration}_Date208c",
                 f"208It{iteration}_Date208cErr", f"208It{iteration}_AgeDiffCorr"]
    )
    
    # Print information about the current iteration
    print(f'\n\033[1m207Pb Correction - Iteration {iteration} Complete\033[0m')
    print('Date of grain in last row (Ma):', current_iteration_results[-1, 5])
    
    # Append the DataFrame to the list
    dfs208_list.append(df_iteration)
    
    # Check if the condition is met for all rows
    if iteration > 1:
        date_difference = abs(current_iteration_results[:, 5] - dfs208_list[-2].iloc[:, 5])
        if all(date_diff < threshold for date_diff in date_difference):
            break  # Exit the loop if the condition is met for all rows

    # Update the date_estimate for the next iteration
    date_estimate = current_iteration_results[:, 5]

    # Increment the iteration counter
    iteration += 1
    
    
# Display the DataFrames
print('\n\n *Preview of results from every iteration:* \n')
for i, result208_df in enumerate(dfs208_list, start=1):
    print(f'\n\n\033[1m207Pb Correction - Iteration {i} Results DataFrame:\033[0m')
    print('\ncolumns: 1- 208/206 common, 2- 208/206* radiogenic, 3- Fraction common Pb (f206), 4- 206*/238 ratio, 5- Total 206 Date (Ma), 6- 208Pb-corrected date (Ma), 7- 208Pb-corrected date uncertainty (2s abs), 8- Difference in 208Pb corr from total206 dates (Ma) \n\n')
    print(result208_df.to_string(max_rows=15))

# Concatenate the first and last iterations into a new DataFrame
result208_df = pd.concat([df] + [dfs208_list[0], dfs208_list[-1]], axis=1)
    

## 2.3 Save 208Pb Correction Results
Run the cell below to concatenate the input table and 208Pb correction tables, then save to Excel file

*Note: Saves most recently run 208Pb correction results from either the manual iteration ([Section 2.2.1](#208Manual)) or threshold iteration ([Section 2.2.2](#208Threshold)). Does not save both. The exported table includes all iteration results from manual iteration OR only the first and last iterations from the threshold iteration.*

In [ ]:
### Combine 208Pb Correction iteration outputs into final table, concatenate with input table
### Will be prompted whether want to save file yes/no below, then window will open to save

# Export the DataFrame to Excel
user_input = input("\033[1mDo you want to save to Excel? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n Either way, will display DataFrame \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path, "OutputFile_208Correction_v1.xlsx"),
        filetypes=["*.xlsx"],
        title="Select Folder and File Name to Save Excel File"
    )

    # If the user selected a file path, save the DataFrame to Excel
    if file_path:
        result208_df.to_excel(file_path, index=False)
        print("\033[1mDataFrame saved to Excel:\033[0m", file_path)
    else:
        print("\033[1mDataFrame not saved.\033[0m")
else:
    print("\033[1mDataFrame not saved.\033[0m")


# Display the concatenated DataFrame 
print('\n\n\033[1mConcatenated DataFrame with 208Pb Correction Results:')
pd.set_option('display.max_columns', None)
result208_df.head(15)

<a id='Section3'></a>
# 3. 207 Pb Correction
Iteratively calculate 207Pb-corrected age

The 207Pb correction method is based on a linear regression of 207Pb/206Pb and 238U/206Pb in Tera-Wasserburg space (Tera and Wasserburg, 1972) along a two-component mixing line between non-radiogenic and radiogenic Pb (Faure, 1986). The 207Pb correction method assumes U-Pb concordance and no Pb loss but, unlike the 208Pb correction, does not assume U/Th remained closed. Because Pb loss is not considered, all corrected dates are (possibly) minimum ages. 

The 207Pb correction iterates on initial age and Stacey and Kramers (1975) terrestrial Pb evolution model. The measured isotopes are the iolite baseline-corrected CPS values (defined in [Section 1](#DefineVariables)). The initial age is the uncorrected date, which is then replaced by the result of subsequent iterations. For the 2 stage terrestrial Pb evolution model of Stacey and Kramers (1975, EPSL), 3700 Ma is the boundary between the first and second stages. If the initial date is > 4570 Ma, it is set to 4750 Ma.

The 207Pb correction is discussed extensively in the literature: Faure (1986), Williams (1997), Ludwig (1998), see also Chew et al. (2011), Thomson et al. (2012), Smye and Stockli (2014), and Vermeesch (2020). The equations below are modified from the Excel routine of Stuart Thomson and benefitted from discussions with Drew Levy, Lisa Stockli, and Zach Foster-Beril.

## 3.1 Define 207Pb Correction Function

In [ ]:
# Define function for 207 Pb correction
# Only need to run this once per session

# 207 Pb Correction
def iterative_207Pb(initial_age, i638_values, i638err_values):
    result_array = np.zeros((len(initial_age), 6))

    # Iterative section for 207Pb Correction
    for i in range(len(initial_age)):
        # Calculate initial common Pb section
        # 1st stage Pb evolution from Stacey & Kramers 1975 EPSL (Table 8)
        if (initial_age[i] != 0.0) & (~np.isnan(initial_age[i])): # Check for NaN or zero values in initial_age
            if initial_age[i] > 3700:
                a64 = 7.19 * (np.exp(lambda238 * 4.570E9) - np.exp(lambda238 * initial_age[i] * 1000000)) + 9.307
                a74 = (7.19 / rat85) * (np.exp(lambda235 * 4.570E9) - np.exp(lambda235 * initial_age[i] * 1000000)) + 10.294
                a76 = a74 / a64 # 207/206 common
            else:
                # 2nd stage Pb evolution from Stacey & Kramers 1975 EPSL (Eqns 5-6; Table 8); see also McLean et al 2011 G3 (Eqns A28-A30); and Vermeesch 2020 Geochron (Eqns 25-26)
                a64 = 9.74 * (np.exp((lambda238 * 3.7E9)) - np.exp((lambda238 * initial_age[i] * 1000000))) + 11.152 # 206/204 common ratio from estimated dates younger than 3.7 Ga using Stacey-Kramers eqn. and initial 206/238 Age estimate
                a74 = (9.74 / rat85) * (np.exp((lambda235 * 3.7E9)) - np.exp((lambda235 * initial_age[i] * 1000000))) + 12.998 # 207/204 common ratio from estimated dates younger than 3.7 Ga using Stacey-Kramers eqn. and initial 206/238 Age estimate
                a76 = a74 / a64 # 207/206 common
        else:
            pass

        # 207 Pb correction
        if i638[i] != 0.0: # Check for zero values in i638[i]
            # calculate expected radiogenic Pb ratios: 207Pb* / 206Pb* = (235/238 U ratio) [(eλ235 t* -1) / (eλ238 t* -1)] 
            cc = (np.exp(lambda235 * (initial_age[i] * 1000000))) - 1 # 207Pb produced from initial age estimate
            dd = (np.exp(lambda238 * (initial_age[i] * 1000000))) - 1 # 206Pb produced from initial age estimate
            c76Pb = rat58 * (cc / dd) # expected radiogenic Pb 207Pb* / 206Pb*

            # calculate f206, fraction of common Pb
            gg = R[i, 4] # recall measured 207/206 Pb ratio calculated from CPS
            f206_207 = (gg - c76Pb) / (a76 - c76Pb) # f206 = (meas. - expected)/(common - expected)
            Rad638_207 = (1 - f206_207) * i638[i] # Radiogenic component, 206*/238 ratio 

            # solve age equation with corrected radiogenic 206*/238 ratio to get the corrected age: t = ln ( 206/238 + 1 ) / λ238
            CorrDate_207 = ((np.log(np.abs(Rad638_207) + 1)) / lambda238) / 1000000 # Corrected date

            # calculate 2SE abs error of 208corrected age using initial % error
            pp = i638[i] # recall initial Final 206/238 ratio
            qq = i638err[i] #recall initial Final 206/238 error 2SE abs
            CorrDateErr_207 = CorrDate_207 / 100 * (qq / pp * 100) # calculate 2SE abs error of corrected age using initial % error

           # Store results in the array
            result_array[i, :] = [a76, c76Pb, f206_207, Rad638_207, CorrDate_207, CorrDateErr_207]

        else:
            pass # Handle the case where i638[i] is zero

    return result_array


## 3.2 Perform Iterative 207Pb Correction
<a id='207Manual'></a>
### 3.2.1 Iterative 207Pb correction - the number of iterations are defined manually

At least 5 iterations is recommended. Chew et al. (2011) demonstrate that the choice of initial age results in a < 0.05% difference in the final 207Pb-corrected age after 5 iterations.

In [ ]:
# Iterative 207Pb correction 

# Define number of iterations
num_iterations = 5

# Initialize a list to store DataFrames
dfs207_list = []

# Initial date estimate
date_estimate = t638 # iteration 1 uses uncorrected date, subsequent iterations use date calculated in previous iteration
date_estimate = np.where(date_estimate > 4570, 4570, date_estimate)

# Perform iterations
for iteration in range(1, num_iterations + 1):
    # Perform 207Pb correction for the current iteration
    current_iteration_results = iterative_207Pb(date_estimate, i638, i638err)
    
    # Create a DataFrame for the current iteration results
    df_iteration = pd.DataFrame(
        current_iteration_results,
        columns=[f"207It{iteration}_207/206c", f"207It{iteration}_207/206*",
                 f"207It{iteration}_f206", f"207It{iteration}_206*/238",
                 f"207It{iteration}_Date207c", f"207It{iteration}_Date207cErr"]
    )
    
    # Print information about the current iteration
    print(f'\n\033[1m207Pb Correction - Iteration {iteration} Complete\033[0m')
    print('Date of grain in last row (Ma):', current_iteration_results[-1, 4])
    
    # Append the DataFrame to the list
    dfs207_list.append(df_iteration)
    
    # Update the date_estimate for the next iteration
    date_estimate = current_iteration_results[:, 4]

# Display the DataFrames
print('\n\n *Preview of results from every iteration:* \n')
for i, result207_df in enumerate(dfs207_list, start=1):
    print(f'\n\033[1m207Pb Correction - Iteration {i} Results DataFrame:\033[0m')
    print('columns: 1- 207/206 common, 2- 207/206* radiogenic, 3- Fraction common Pb (f206), 4- 207Pb-corrected date (Ma), 5- 207Pb-corrected date uncertainty (2s abs)\n\n')
    print(result207_df.to_string(max_rows=15))
    
# Combine DataFrames along the second axis (columns)
# Here, includes all iterations
result207_df = pd.concat([df] + dfs207_list, axis=1)

<a id='207Threshold'></a>
### 3.2.2 Iterative 207Pb correction - the number of iterations set by threshold

Loops iterations until either the threshold or num_iterations is reached. The threshold is defined as the percent difference in the corrected date between the current and previous iteration (i.e., 0.05%). If the date difference is above the threshold for any grain, another iteration will run for all grains.

*Note: All iteration results are displayed but only the first and last are included in the export table (Sections 3.3 and 3.4)*

In [ ]:
# Try iteration with threshold

# Define the number of iterations
num_iterations = 200  # Set a large number to ensure it runs until the threshold is met, may need to increase

# Specify the threshold
threshold = 1 / 100  # 0.05% expressed as a decimal

# Initialize a list to store DataFrames
dfs207_list = []

# Initial date estimate
date_estimate = t638  # iteration 1 uses uncorrected date, subsequent iterations use date calculated in the previous iteration
date_estimate = np.where(date_estimate > 4570, 4570, date_estimate)

# Initialize iteration counter
iteration = 1

# Perform iterations
while iteration <= num_iterations:
    # Perform 207Pb correction for the current iteration
    current_iteration_results = iterative_207Pb(date_estimate, i638, i638err)

    # Create a DataFrame for the current iteration results
    df_iteration = pd.DataFrame(
        current_iteration_results,
        columns=[f"207It{iteration}_207/206c", f"207It{iteration}_207/206*",
                 f"207It{iteration}_f206", f"207It{iteration}_206*/238",
                 f"207It{iteration}_Date207c", f"207It{iteration}_Date207cErr"]
    )

    # Print information about the current iteration
    print(f'\n\033[1m207Pb Correction - Iteration {iteration} Complete\033[0m')
    print('Date of grain in the last row (Ma):', current_iteration_results[-1, 4])

    # Append the DataFrame to the list
    dfs207_list.append(df_iteration)

    # Check if the condition is met for all rows
    if iteration > 1:
        date_difference = abs(current_iteration_results[:, 4] - dfs207_list[-2].iloc[:, 4])
        if all(date_diff < threshold for date_diff in date_difference):
            break  # Exit the loop if the condition is met for all rows

    # Update the date_estimate for the next iteration
    date_estimate = current_iteration_results[:, 4]

    # Increment the iteration counter
    iteration += 1

# Display the DataFrames
print('\n\n *Preview of results from every iteration:* \n')
for i, result207_df in enumerate(dfs207_list, start=1):
    print(f'\n\033[1m207Pb Correction - Iteration {i} Results DataFrame:\033[0m')
    print('columns: 1- 207/206 common, 2- 207/206* radiogenic, 3- Fraction common Pb (f206), 4- 207Pb-corrected date (Ma), 5- 207Pb-corrected date uncertainty (2s abs)\n\n')
    print(result207_df.to_string(max_rows=15))

# Concatenate the first and last iterations into a new DataFrame
result207_df = pd.concat([df] + [dfs207_list[0], dfs207_list[-1]], axis=1)
    

## 3.3 Save 207Pb Correction Results
Run the cell below to concatenate the input table and 207Pb-correction tables, then save to Excel file.

*Note: Saves most recently run 208Pb correction results from either the manual iteration ([Section 3.2.1](#207Manual)) or threshold iteration ([Section 3.2.2](#207Threshold)). Does not save both. The exported table includes all iteration results from manual iteration OR only the first and last iterations from the threshold iteration.*

The 207Pb correction results can be used to calculate discordance in the notebook UPb-Plotter

In [ ]:
### Combine 207Pb Correction iteration outputs into final table, concatenate with input table
### Will be prompted whether want to save file yes/no below, then window will open to save


# Export the DataFrame to Excel
user_input = input("\033[1mDo you want to save to Excel? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n Either way, will display DataFrame \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path, "OutputFile_207Correction_v1.xlsx"),
        filetypes=["*.xlsx"],
        title="Select Folder and File Name to Save Excel File"
    )

    # If the user selected a file path, save the DataFrame to Excel
    if file_path:
        result207_df.to_excel(file_path, index=False)
        print("\033[1mDataFrame saved to Excel:\033[0m", file_path)
    else:
        print("\033[1mDataFrame not saved.\033[0m")
else:
    print("\033[1mDataFrame not saved.\033[0m")


# Display the concatenated DataFrame 
pd.set_option('display.max_columns', None)
print('\n\n\033[1mConcatenated DataFrame with 207Pb Correction Results:')
result207_df.head(15)

<a id='Section4'></a>
# 4. Export Combined 208Pb and 207Pb Correction Results
Run the cell below to concatenate the input table, 208Pb-correction tables and 207Pb-correction tables, then save to Excel file

Note: Saves the 208Pb and 207Pb correction tables most recently run, *not both* the manual and threshold iteration tables.

In [ ]:
### Combine 207Pb Correction iteration outputs into final table, concatenate with input table
### Will be prompted whether want to save file yes/no below, then window will open to save

# Combine DataFrames along the second axis (columns)
result208207_df = pd.concat([df] + dfs208_list + dfs207_list, axis=1)


# Export the DataFrame to Excel
user_input = input("\033[1mDo you want to save to Excel? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n Either way, will display DataFrame \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path, "OutputFile_CombinedPbCorrection_v1.xlsx"),
        filetypes=["*.xlsx"],
        title="Select Folder and File Name to Save Excel File"
    )

    # If the user selected a file path, save the DataFrame to Excel
    if file_path:
        result208207_df.to_excel(file_path, index=False)
        print("\033[1mDataFrame saved to Excel:\033[0m", file_path)
    else:
        print("\033[1mDataFrame not saved.\033[0m")
else:
    print("\033[1mDataFrame not saved.\033[0m")


# Display the concatenated DataFrame 
pd.set_option('display.max_columns', None)
print('\n\n\033[1mConcatenated DataFrame with 207Pb Correction Results:')
result207_df.head(15)